In [1]:
import os
import json
!pip install spotipy
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt

In [2]:
#The Goal: 
# A supervised learning algorithm, such as a decision tree or support vector machine, 
# could be used to classify the genre of music based on the data provided. 
# The algorithm would be trained on a dataset of labeled music genres and the associated data values, 
# and then it could be used to predict the genre of music given a new set of data values.

cid ='d8762235995b4ddcbc0df90739df9a63' # Client ID; copy this from your app created on beta.developer.spotify.com
secret = '887a4eccdbde4195bfc0f7f4c472fe70' # Client Secret; copy this from your app
username = '12155311349' # Your Spotify username

#for avaliable scopes see https://developer.spotify.com/web-api/using-scopes/
scope = 'user-library-read playlist-modify-public playlist-read-private'

redirect_uri='https://localhost:8080/callback/'

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret) 

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

token = util.prompt_for_user_token(username,scope,client_id=cid,client_secret=secret,redirect_uri="https://localhost:8080/callback/")

if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)


In [3]:
#Creating function and having user give PlaylistID, and genre (0 = counrty, 1 = rock...)
def input_playlist(playlistID, genre): 

    playlist = sp.user_playlist(username, playlistID);

    tracks = playlist["tracks"];
    songs = tracks["items"];
    total_tracks = tracks["total"]
    
    # Iterate over all tracks in the playlist using offset to get beyond the first 100 tracks
    for i in range(100, total_tracks, 100):
        tracks = sp.user_playlist_tracks(username, playlistID, offset=i)
        songs += tracks["items"]
        
    #Putting Track ID, Track Name, Album, and artist name in matrix's

    track_ids = [] 
    track_names = []
    album = []
    artist =[]

    #Getting Track Data
    for i in range(0, len(songs)):
        if songs[i]['track']['id'] != None: # Removes the local tracks in your playlist if there is any
            track_ids.append(songs[i]['track']['id'])
            track_names.append(songs[i]['track']['name'])
            album.append(songs[i]['track']['album']['name'])
            artist.append(songs[i]['track']['artists'][0]['name'])

    features = []
    for i in range(0,len(songs)):
        audio_features = sp.audio_features(track_ids[i])
        for track in audio_features:

          if track is None:
            print(track)
            features.append({'danceability': 0, 'energy': 0, 'key': 0, 'loudness': 0, 'mode': 0, 'speechiness': 0, 'acousticness': 0, 'instrumentalness': 0, 'liveness': 0, 'valence': 0, 'tempo': 0, 'type': 'audio_features', 'id': '00000', 'uri': 'spotify:track:0', 'track_href': 'https://api.spotify.com/', 'analysis_url': 'https://api.spotify.com/', 'duration_ms': 0, 'time_signature': 0})
          else:
            features.append(track)

    #Creating our DataFrames
    df = pd.DataFrame(features, index = track_names) 

    #Adding a new column
    df['genre'] = genre #Country
    df['Song Title'] = track_names
    df['Artist'] = artist

    # Scale between 0 and 1
    df["key"] = (df["key"] / df["key"].max())
    df["tempo"] = (df["tempo"] / df["tempo"].max())
    df["loudness"] = (df["loudness"] / df["loudness"].min())
    
    #calculate the IQR for instrumentalness and acousticness
    Q1 = df["instrumentalness"].quantile(0.25)
    Q3 = df["instrumentalness"].quantile(0.75)
    IQR = Q3 - Q1
    
    Q12 = df["acousticness"].quantile(0.25)
    Q32 = df["acousticness"].quantile(0.75)
    IQR2 = Q32 - Q12
    
    #Remove rows with outliers
    df = df[~((df["instrumentalness"] < (df["instrumentalness"].quantile(0.25) - 1.5 * IQR)) | (df["instrumentalness"] > (df["instrumentalness"].quantile(0.75) + 1.5 * IQR)))]
    df = df[~((df["acousticness"] < (df["acousticness"].quantile(0.25) - 1.5 * IQR2)) | (df["acousticness"] > (df["acousticness"].quantile(0.75) + 1.5 * IQR2)))]
    
#     Q1 = df.quantile(0.25)
#     Q3 = df.quantile(0.75)
#     IQR = Q3 - Q1
#     df = df[~((df.sub(Q1, axis=1).div(IQR, axis=1)).lt(-1.5) | (df.sub(Q1, axis=1).div(IQR, axis=1)).gt(1.5)).any(axis=1)]
    return df

In [4]:
#userPlaylistID = 'https://open.spotify.com/playlist/5PlFP6HbcJKMqAZ557UgSG?si=02a1a81194874f32'
# # userPlaylistID = 'https://open.spotify.com/playlist/37i9dQZF1DWXRqgorJj26U?si=7630a4c40aea443e' #Rock
# userPlaylistID= 'https://open.spotify.com/playlist/4P3NZ1qRZn9eDLpy59YOdj?si=da0a03bf133e469f' #Retro Playlist
# userPlaylistID = 'https://open.spotify.com/playlist/37i9dQZF1DWXRqgorJj26U?si=4d36865c93204d3b'
#userPlaylistID = 'https://open.spotify.com/playlist/3DygxluFugJu7zVAjsMWgb?si=9499fee0f7204717' #Fort
userPlaylistID = 'https://open.spotify.com/playlist/292reS0h3C44giIljfyuMz?si=9871556160d64d84' #Laurens Playlist
user_playlist = input_playlist(userPlaylistID,'none')
num_user_playlist = user_playlist[['danceability','energy','key','loudness','mode','speechiness','acousticness','instrumentalness','liveness','valence','tempo','time_signature','genre',]]
num_user_playlist = num_user_playlist.drop('genre', axis=1)

In [5]:
num_user_playlist

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
"Pills & Automobiles (feat. Yo Gotti, A Boogie Wit Da Hoodie & Kodak Black)",0.820,0.696,0.636364,0.382188,0,0.1820,0.11900,0.000000,0.2320,0.6170,0.754720,4
Cookie,0.652,0.758,0.727273,0.368122,0,0.0315,0.12800,0.000000,0.2730,0.4860,0.681560,4
Mirror,0.586,0.706,0.000000,0.393224,0,0.0787,0.08940,0.000000,0.0772,0.1360,0.911192,4
Play,0.691,0.640,0.181818,0.498057,0,0.0425,0.05010,0.000000,0.1170,0.4310,0.509919,4
B.E.D.,0.739,0.536,0.090909,0.505207,1,0.0587,0.00368,0.000000,0.0968,0.1930,0.635204,4
Up Down (Do This All Day) (feat. B.o.B),0.763,0.530,0.454545,0.431613,0,0.0577,0.27200,0.000000,0.1600,0.4690,0.505120,4
The Weekend,0.400,0.366,0.727273,0.730883,0,0.0597,0.24200,0.000000,0.1260,0.1610,0.381642,3
No Guidance (feat. Drake),0.696,0.454,0.090909,0.525023,0,0.1530,0.12300,0.000000,0.1550,0.1360,0.483751,4
Post to Be (feat. Chris Brown & Jhene Aiko),0.733,0.676,0.909091,0.439462,0,0.0432,0.06970,0.000000,0.2080,0.7010,0.507280,4
"Or Nah (feat. The Weeknd, Wiz Khalifa & DJ Mustard) - Remix",0.805,0.330,0.090909,0.677028,0,0.1000,0.22000,0.000000,0.1050,0.2110,0.634933,4


In [6]:
#Data has already been created and saved. Check how it is done in AI_Random_Forest to see where the data comes from. It's necessary to save time. 
full_data = pd.read_csv('full_data.csv')
full_data_random = pd.read_csv('full_data_random.csv')
num_data_random = pd.read_csv('num_data_random.csv')

In [7]:
country = full_data_random[full_data_random['genre'] == 'country']
rock = full_data_random[full_data_random['genre'] == 'rock']
rap = full_data_random[full_data_random['genre'] == 'rap']
rnb = full_data_random[full_data_random['genre'] == 'rnb']
pop = full_data_random[full_data_random['genre'] == 'pop']
classical = full_data_random[full_data_random['genre'] == 'classical']

In [8]:
# Load the previously trained model
import pickle
with open('random_forest_model.pkl', 'rb') as f:
    model = pickle.load(f)
# Load the new data

# Predict the class labels of the new data
predicted_genre = model.predict(num_user_playlist)

# Add the predicted genre to the dataframe
num_user_playlist['predicted_genre'] = predicted_genre

# Print the dataframe
num_user_playlist 

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,predicted_genre
"Pills & Automobiles (feat. Yo Gotti, A Boogie Wit Da Hoodie & Kodak Black)",0.820,0.696,0.636364,0.382188,0,0.1820,0.11900,0.000000,0.2320,0.6170,0.754720,4,rap
Cookie,0.652,0.758,0.727273,0.368122,0,0.0315,0.12800,0.000000,0.2730,0.4860,0.681560,4,country
Mirror,0.586,0.706,0.000000,0.393224,0,0.0787,0.08940,0.000000,0.0772,0.1360,0.911192,4,pop
Play,0.691,0.640,0.181818,0.498057,0,0.0425,0.05010,0.000000,0.1170,0.4310,0.509919,4,pop
B.E.D.,0.739,0.536,0.090909,0.505207,1,0.0587,0.00368,0.000000,0.0968,0.1930,0.635204,4,rnb
Up Down (Do This All Day) (feat. B.o.B),0.763,0.530,0.454545,0.431613,0,0.0577,0.27200,0.000000,0.1600,0.4690,0.505120,4,rnb
The Weekend,0.400,0.366,0.727273,0.730883,0,0.0597,0.24200,0.000000,0.1260,0.1610,0.381642,3,rnb
No Guidance (feat. Drake),0.696,0.454,0.090909,0.525023,0,0.1530,0.12300,0.000000,0.1550,0.1360,0.483751,4,rap
Post to Be (feat. Chris Brown & Jhene Aiko),0.733,0.676,0.909091,0.439462,0,0.0432,0.06970,0.000000,0.2080,0.7010,0.507280,4,pop
"Or Nah (feat. The Weeknd, Wiz Khalifa & DJ Mustard) - Remix",0.805,0.330,0.090909,0.677028,0,0.1000,0.22000,0.000000,0.1050,0.2110,0.634933,4,rap


In [9]:
# predicted.to_csv('AI_Results.csv', index=False)
print('Total Songs: ', num_user_playlist.index.size)
print('Country: ',  num_user_playlist[num_user_playlist['predicted_genre'] == 'country'].index.size)
print('Rock: ',  num_user_playlist[num_user_playlist['predicted_genre'] == 'rock'].index.size)
print('Rap: ',  num_user_playlist[num_user_playlist['predicted_genre'] == 'rap'].index.size)
print('RnB: ',  num_user_playlist[num_user_playlist['predicted_genre'] == 'rnb'].index.size)
print('Pop: ',  num_user_playlist[num_user_playlist['predicted_genre'] == 'pop'].index.size)
print('Classical: ',  num_user_playlist[num_user_playlist['predicted_genre'] == 'classical'].index.size)


Total Songs:  55
Country:  2
Rock:  0
Rap:  17
RnB:  22
Pop:  14
Classical:  0


In [10]:
# num_user_playlist[num_user_playlist['danceability'] == .736]